In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import seaborn as sns
import numpy as np#import basic library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential


In [3]:
df = pd.read_excel("/content/drive/MyDrive/DataSet/apy-milletstats.xlsx")
df.head()

,Crop,Year,State,Area ('000 Hectares),Production ('000 Tonnes),Yield (Kg per Ha))
0,Finger Millet (Ragi),1966-67,All India,1984.2,1630.6,821.792158
1,Finger Millet (Ragi),1967-68,All India,2291.2,1884.2,822.363827
2,Finger Millet (Ragi),1968-69,All India,2238.2,1648.0,736.305960
3,Finger Millet (Ragi),1969-70,All India,2783.4,2117.2,760.652439
4,Finger Millet (Ragi),1970-71,All India,2472.4,2155.0,871.622715


In [4]:
df.shape

(4796, 6)

In [5]:
df.duplicated().sum()

0

In [6]:
df.dtypes

Crop                         object
Year                         object
State                        object
Area ('000 Hectares)         object
Production ('000 Tonnes)    float64
Yield (Kg per Ha))          float64
dtype: object

In [7]:
df.isnull().sum()/df.shape[0]*100

Crop                         0.000000
Year                         0.000000
State                        0.000000
Area ('000 Hectares)        12.572977
Production ('000 Tonnes)    13.448707
Yield (Kg per Ha))          14.282736
dtype: float64

In [8]:
df["Area ('000 Hectares)"] = df["Area ('000 Hectares)"].replace(".", np.nan, regex=True)

In [9]:
from numpy import nan
df["Area ('000 Hectares)"].replace(np.nan,0.0,inplace=True)


In [10]:
df["Area ('000 Hectares)"].astype('float64')

0       1984.200
1       2291.200
2       2238.200
3       2783.400
4       2472.400
          ...   
4791       0.040
4792       0.026
4793       0.040
4794       0.064
4795       0.180
Name: Area ('000 Hectares), Length: 4796, dtype: float64

In [11]:
df.dtypes

Crop                         object
Year                         object
State                        object
Area ('000 Hectares)        float64
Production ('000 Tonnes)    float64
Yield (Kg per Ha))          float64
dtype: object

In [12]:
a=df["Area ('000 Hectares)"].mean()
df["Area ('000 Hectares)"].replace(0.0,a,inplace=True)

In [13]:
b=df["Production ('000 Tonnes)"].mean()
df["Production ('000 Tonnes)"].fillna(a,inplace=True)

In [14]:
b=df["Yield (Kg per Ha))"].mean()
df["Yield (Kg per Ha))"].fillna(a,inplace=True)

In [15]:
df.isnull().sum()/df.shape[0]*100

Crop                        0.0
Year                        0.0
State                       0.0
Area ('000 Hectares)        0.0
Production ('000 Tonnes)    0.0
Yield (Kg per Ha))          0.0
dtype: float64

In [16]:
#separate object type columns and numeric type column and hold on new dataframe
df_cat=df.select_dtypes(object)
df_cat.dtypes

Crop     object
Year     object
State    object
dtype: object

In [17]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
for col in df_cat:
  df_cat[col]=le.fit_transform(df_cat[col])


In [18]:
df_num=df.select_dtypes(['int64','float64'])
df_num.dtypes

Area ('000 Hectares)        float64
Production ('000 Tonnes)    float64
Yield (Kg per Ha))          float64
dtype: object

In [19]:
df_new=pd.concat([df_num,df_cat],axis=1)
df_new.head()

,Area ('000 Hectares),Production ('000 Tonnes),Yield (Kg per Ha)),Crop,Year,State
0,1984.2,1630.6,821.792158,0,0,0
1,2291.2,1884.2,822.363827,0,1,0
2,2238.2,1648.0,736.305960,0,2,0
3,2783.4,2117.2,760.652439,0,3,0
4,2472.4,2155.0,871.622715,0,4,0


In [20]:
x=df_new.drop("Yield (Kg per Ha))",axis=1)
y=df_new["Yield (Kg per Ha))"]

In [21]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.3,random_state=1)

In [22]:
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
xtrain=ss.fit_transform(xtrain)
xtest=ss.transform(xtest)

In [33]:
from tensorflow.keras.callbacks import EarlyStopping

model=Sequential()

model.add(Dense(256, input_dim=5, activation="relu"))
model.add(Dense(256, activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(32, activation="relu"))
model.add(Dense(16, activation="relu"))
model.add(Dense(8, activation="relu"))
model.add(Dense(1))

model.compile(optimizer="adam",loss="mse") #complile model
earlystop=EarlyStopping(monitor="val_loss",mode="min",verbose=1,patience=50) #use early stopping

model.fit(xtrain,ytrain,epochs=500,validation_data=(xtest,ytest),callbacks=[earlystop], batch_size=64) #train data


Epoch 1/500
53/53 [==============================] - 4s 34ms/step - loss: 182784180224.0000 - val_loss: 98451824640.0000
Epoch 2/500
53/53 [==============================] - 1s 15ms/step - loss: 182766370816.0000 - val_loss: 98400018432.0000
Epoch 3/500
53/53 [==============================] - 1s 15ms/step - loss: 182470852608.0000 - val_loss: 97951645696.0000
Epoch 4/500
53/53 [==============================] - 0s 9ms/step - loss: 181728690176.0000 - val_loss: 97478082560.0000
Epoch 5/500
53/53 [==============================] - 0s 8ms/step - loss: 180391772160.0000 - val_loss: 97069187072.0000
Epoch 6/500
53/53 [==============================] - 0s 8ms/step - loss: 179183452160.0000 - val_loss: 96220135424.0000
Epoch 7/500
53/53 [==============================] - 0s 8ms/step - loss: 177015209984.0000 - val_loss: 95574269952.0000
Epoch 8/500
53/53 [==============================] - 0s 8ms/step - loss: 174165245952.0000 - val_loss: 93398622208.0000
Epoch 9/500
53/53 [==================

In [34]:
from sklearn.metrics import mean_squared_error, r2_score
ypred=model.predict(xtest)
print("MSE",mean_squared_error(ytest,ypred))
print("R2 Score",r2_score(ytest,ypred))

45/45 [==============================] - 0s 2ms/step
MSE 23466190875.52165
R2 Score 0.7612287243244102


In [38]:
from tensorflow.keras.models import load_model
model.save("millets_model")
storemodel=load_model("millets_model")

